In [1]:
# Modify the file A00_setup.  Note the working directory is changed
import os
os.chdir(os.getcwd().rsplit(os.path.sep + 'code')[0] + os.path.sep + 'code')
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./A_target_count_encoding/A00_setup.py").load_module()

# XGBoost, Modified Hierarchical - Loop Midpoint, Remove Low Volume
Regular hierarchical encoding but different amounts of blending, remove low-volume codes

*This script takes about 5 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import pickle

In [3]:
from pathlib import Path

In [4]:
import sklearn as sk
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance, partial_dependence
from scikitplot.metrics import plot_lift_curve

In [5]:
from sba_gnn.sba_gnn import sg_plot, sg_blender
from sba_gnn.sba_gnn.sg_blender import HierarchicalEncoder # Target encoder

## Import Processed Datasets

In [6]:
sba_loans = pd.read_parquet(Path(setup.parent_path).joinpath('01_DATA_transformed.parquet')) 

In [7]:
predictor_features = setup.predictor_features + ['mhier_NAICS_alt']
print(predictor_features)
with open(Path(setup.temp_path).joinpath('13_DATA_features_predict.pkl'), 'wb') as fout:
    pickle.dump(predictor_features, fout)

['NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business', 'urban_flag', 'franchise_flag', 'mhier_NAICS_alt']


In [8]:
best_params_df = pd.read_csv(Path(setup.parent_path).joinpath('03_REPORT_fit_parameter_selected.csv'))
best_params = best_params_df.to_dict(orient='records')[0]
best_params

{'subsample': 0.8,
 'scale_pos_weight': 1.0,
 'reg_lambda': 0.01,
 'reg_alpha': 0.001,
 'min_child_weight': 50,
 'max_depth': 8,
 'learning_rate': 0.1,
 'gamma': 2}

In [9]:
xgb_params = dict(best_params, 
                   **{'objective':"binary:logistic", 'n_estimators':setup.xgb_n_estimators})

In [10]:
train_counts = pd.read_parquet(Path(setup.temp_path).joinpath('A60_DATA_train_counting.parquet')) 

## Function to get metrics

In [11]:
def get_metrics(data, lambda_k = 100, enc_features = ['NAICS', 'NAICS_sector']):
    
    # Alternative encoding
    train_df = data[data['dset'] == 'train']
    hier_enc = HierarchicalEncoder(lambda_k = lambda_k)
    hier_enc.fit(train_df[enc_features],train_df['target'])
    hier_col = hier_enc.transform(data[enc_features]) \
        .rename('mhier_NAICS_alt')
    
    # Append to data
    data = pd.concat([data.drop(columns='mhier_NAICS_alt', errors='ignore'), 
                       hier_col], axis=1)
    train_df = data[data['dset'] == 'train']
    test_df = data[data['dset'] == 'test']
    val_df = data[data['dset'] == 'val']
    
    # Remove low volume codes from train only
    counts_thresh = train_counts[train_counts['count'] > lambda_k][['NAICS']]
    train_df = train_df.merge(counts_thresh, on='NAICS')
    
    # Model fit
    xgb_model = xgb.XGBClassifier(objective="binary:logistic",
                             n_estimators =  setup.xgb_n_estimators)
    xgb_model = xgb_model.set_params(**xgb_params)
    xgb_model.fit(train_df[predictor_features], train_df['target'],
              eval_set = [(train_df[predictor_features], train_df['target']),
                         (val_df[predictor_features], val_df['target'])])
    
    # Predict
    pred_df = pd.concat([data[['LoanNr_ChkDgt', 'dset', 'target', 'dset_naics_holdout']].reset_index(),
                     pd.DataFrame({'predict_prob':
                                   xgb_model.predict_proba(data[predictor_features])[:,1]})],
                    axis=1) \
    .set_index('index')
    
    # Decision threshold
    pred_train_df = pred_df[pred_df['dset'] == 'train']
    thresh_tune_data = sg_plot.get_f1_frame(pred_train_df['target'], 
                                        pred_train_df['predict_prob'])
    thresh_head = thresh_tune_data.sort_values('f1', ascending=False).head(2)
    best_thresh = thresh_head['thresh'].iloc[0]
    pred_df['predict_bin'] = np.where(pred_df['predict_prob'] >= best_thresh, 1, 0)
    
    # Metrics
    metrics_dset_df = pred_df.groupby('dset') \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_test_df = pred_df[pred_df['dset'] == 'test'] \
        .groupby(['dset', 'dset_naics_holdout']) \
        .apply(lambda x: sg_plot.dset_metrics(x.target, x.predict_bin, x.predict_prob)) \
        .reset_index()
    metrics_df = pd.concat([metrics_dset_df, metrics_test_df])
    
    return pred_df, metrics_df

## Loop to get predictions, metrics

In [12]:
count_sets = [5, 10, 20, 50, 100, 200, 500, 1000, 2000]

In [13]:
%%capture
pred_df = pd.DataFrame()
metrics_df = pd.DataFrame()
for c in count_sets:
    this_pred, this_met = get_metrics(sba_loans, c,
                                      ['NAICS', 'NAICS_5', 'NAICS_4', 'NAICS_3', 'NAICS_sector'])
    this_pred['c'] = c
    pred_df = pd.concat([pred_df, this_pred])
    this_met['c'] = c
    metrics_df = pd.concat([metrics_df, this_met])

In [14]:
metrics_df[metrics_df['dset_naics_holdout'] == 1]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,c
1,test,0.692984,0.427211,0.323166,0.630065,0.366721,0.731799,1.0,5
1,test,0.693169,0.427555,0.323426,0.630573,0.366981,0.731876,1.0,10
1,test,0.696351,0.428625,0.325670,0.626766,0.367464,0.732620,1.0,20
1,test,0.694936,0.427983,0.324589,0.628035,0.367698,0.733377,1.0,50
1,test,0.693814,0.427907,0.323940,0.630150,0.367774,0.734024,1.0,100
1,test,0.693414,0.428702,0.324094,0.633026,0.369080,0.734993,1.0,200
1,test,0.687465,0.424849,0.319152,0.635225,0.366447,0.732541,1.0,500
1,test,0.646483,0.425807,0.302055,0.721343,0.367223,0.732635,1.0,1000
1,test,0.653416,0.421304,0.302406,0.694273,0.355507,0.725495,1.0,2000


In [15]:
metrics_df[metrics_df['dset_naics_holdout'] == 0]

,dset,accuracy_score,f1_score,precision_score,recall_score,average_precision_score,roc_auc_score,dset_naics_holdout,c
0,test,0.659633,0.450251,0.337129,0.677626,0.383197,0.728491,0.0,5
0,test,0.660660,0.449131,0.337141,0.672528,0.383389,0.728534,0.0,10
0,test,0.661120,0.450021,0.337765,0.674036,0.384017,0.728934,0.0,20
0,test,0.661773,0.451242,0.338632,0.676065,0.384425,0.729040,0.0,50
0,test,0.659002,0.448917,0.336226,0.675233,0.383331,0.728528,0.0,100
0,test,0.659151,0.449673,0.336636,0.677002,0.383663,0.728801,0.0,200
0,test,0.656434,0.448619,0.334846,0.679499,0.380865,0.726551,0.0,500
0,test,0.627168,0.447892,0.322039,0.735213,0.377983,0.723629,0.0,1000
0,test,0.623519,0.443274,0.318522,0.728658,0.366603,0.716673,0.0,2000


In [16]:
pred_df.to_parquet(Path(setup.temp_path).joinpath('A61_DATA_combined_predictions.parquet'))
metrics_df.to_csv(Path(setup.temp_path).joinpath('A61_REPORT_metrics.csv'), index=True)